In [9]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
os.chdir("drive/My Drive")


FileNotFoundError: ignored

In [11]:
!ls

 dataset  'dog vs cat.zip'


In [0]:
# only run for first time 
#!mkdir 'image_classifier'

In [0]:
#!unzip dogs-cats-images.zip -d 'image_classifier'

In [14]:
os.chdir('image_classifier/')


FileNotFoundError: ignored

In [0]:
#!unzip dog\ vs\ cat.zip

In [0]:
os.chdir('dataset')

In [16]:
import cv2
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
from tqdm import tqdm
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
%matplotlib inline 

Using TensorFlow backend.


In [0]:
def label_pet_image_one_hot_encoder(img):
    pet = img.split('.')[0]
    if pet == 'cat': return [1,0]
    elif pet == 'dog': return [0,1]
    elif pet =='Unknown': return [0,0]

In [0]:
def load_images_from_folder(folder,isTrain = True):
#     images = []
#     labels = []
    data_df = []
    for filename in tqdm(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            if isTrain:
                label = label_pet_image_one_hot_encoder(filename.split('.')[0])
            else:
                label =label_pet_image_one_hot_encoder('Unknown')
                
            i = cv2.resize(img,(224,224))
            data_df.append([np.array(i),np.array(label)])
    shuffle(data_df)    
          
    return data_df

In [0]:
dogs_train = load_images_from_folder('training_set/dogs')


 14%|█▍        | 566/4001 [04:14<23:36,  2.42it/s]

In [0]:
cats_train = load_images_from_folder('training_set/cats')

In [0]:
dogs_test = load_images_from_folder('test_set/dogs',False)
cats_test = load_images_from_folder('test_set/cats',False)

In [0]:
dogs_train[0][0].shape,cats_train[0][0].shape,\
dogs_test[0][0].shape,cats_test[0][0].shape

In [0]:
len(dogs_train),len(cats_train),len(dogs_test),len(cats_test)

In [0]:
def show_images(data, isTest=False):
    f, ax = plt.subplots(5,5, figsize=(15,15))
    for i,data in enumerate(data[:25]):
        img_num = data[1]
        img_data = data[0]
        label = np.argmax(img_num)
        if isTest ==True:
            str_label="None"


        else:

            if (label  == 1): 
                str_label='Dog'



            elif (label == 0): 
                str_label='Cat'
        ax[i//5, i%5].imshow(img_data,)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))

        
    plt.show()

In [0]:
show_images(cats_train)

In [0]:
show_images(dogs_train)

In [0]:
show_images(dogs_test,True)

In [0]:
show_images(cats_test,True)

In [0]:
X = np.array([i[0] for i in dogs_train+cats_train]).reshape(-1,224,224,3)
y = np.array([i[1] for i in dogs_train+cats_train])

In [0]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='max', weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'))
model.add(Dense(2, activation='softmax'))
# ResNet-50 model is already trained, should not be trained
model.layers[0].trainable = True

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=33)

In [0]:
train_model = model.fit(X_train, y_train,batch_size=16,epochs=10,verbose=1,validation_data=(X_val,y_val))

In [0]:
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(14,6))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend()
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()
plot_accuracy_and_loss(train_model)